# Experiment 5

### Aim: Implement Transfer Learning

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [4]:
# Define data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
train_datagen.fit(train_images)

In [5]:
# Load pre-trained VGG16 model without including top dense layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
# Freeze all layers in base model
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Add custom dense layers for CIFAR-10 classification
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [8]:
# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Train the model
history = model.fit(train_datagen.flow(train_images, train_labels, batch_size=64),
                    steps_per_epoch=len(train_images) / 64, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
781/781 [==============================] - 38s 43ms/step - loss: 1.6069 - accuracy: 0.1017 - val_loss: 1.3236 - val_accuracy: 0.0939
Epoch 2/10
781/781 [==============================] - 32s 41ms/step - loss: 1.4187 - accuracy: 0.0974 - val_loss: 1.2689 - val_accuracy: 0.1079
Epoch 3/10
781/781 [==============================] - 32s 40ms/step - loss: 1.3620 - accuracy: 0.0978 - val_loss: 1.2226 - val_accuracy: 0.0993
Epoch 4/10
781/781 [==============================] - 33s 42ms/step - loss: 1.3305 - accuracy: 0.0972 - val_loss: 1.2217 - val_accuracy: 0.1270
Epoch 5/10
781/781 [==============================] - 31s 40ms/step - loss: 1.3192 - accuracy: 0.0978 - val_loss: 1.2003 - val_accuracy: 0.0874
Epoch 6/10
781/781 [==============================] - 32s 41ms/step - loss: 1.3034 - accuracy: 0.0978 - val_loss: 1.1808 - val_accuracy: 0.1073
Epoch 7/10
781/781 [==============================] - 31s 40ms/step - loss: 1.2891 - accuracy: 0.0978 - val_loss: 1.1794 - val_accuracy:

In [10]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 2s 8ms/step - loss: 1.1607 - accuracy: 0.1063
Test accuracy: 0.1062999963760376
